# **Dhaka AI 2020 Inference Notebook**

This inference notebook was created for the Dhaka AI Traffic Detection Challenge 2020. We use the yolov5 object detection algorithm developed by ultralytics.


---




---


##**Team Name** : TrafficZero
##### **Institution** : Islamic University of Technology

> *Members* :

* Refaat Mohammad Alamgir
* Ali Abir Shuvro
* Mueeze Al Mushabbir 
* Md Ashfaq Raiyan
* Nusrat Jahan Rani



---



---








##**Details of our solution** :

**Model used** - yolov5x [(link to yolov5 repo)](https://github.com/ultralytics/yolov5)


**Datasets used** - 

1.   Training dataset provided by DhakaAI  [[1]](https://doi.org/10.7910/DVN/POREXF)
2.   Poribohon-BD [[2]](https://data.mendeley.com/datasets/pwyyg8zmk5/1)
3.   vehicle-dataset [[3]](https://github.com/kasperhaaland/vehicle-dataset)
4.   Test dataset for round 1 (annotated ourselves after the end of round 1) [[1]](https://doi.org/10.7910/DVN/POREXF)
     

---


---







## 1. Removing folders from previous session

Some folders may persist from previous sessions. We remove them to avoid any unexpected results.

In [ ]:
import os
import shutil
for folder in ['dhakaai','yolov5','sample_data']:
  dirpath = os.path.join('/content', folder)
  if os.path.exists(dirpath) and os.path.isdir(dirpath):
      shutil.rmtree(dirpath)



---


## 2. Setup

Clone repo, install dependencies, `%cd` into `./yolov5` folder and check GPU.

Note - if this block of code throws an error, just restart the runtime.


---



In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla P4', major=6, minor=1, total_memory=7611MB, multi_processor_count=20)




---


## 3. Downloading Test Set 2


*****THE ORGANIZERS CAN ADD THEIR OWN DATA HERE** ***



---



In [ ]:
%cd /content/
!gdown --id '132Ga8DYFQh8-yQIoIP0NI6dQWOAkvuNE'
!unzip test_round_2.zip; rm test_round_2.zip;
clear_output()
!mv /content/test\ round\ 2 /content/test_round_2/



---


## 4. Model Ensembling


---



In [ ]:
# code for downloading previous weights. 

%cd /content/yolov5/
path = os.path.join('/content/yolov5/weights','myweights')
os.mkdir(path)

# **** Round 1 ****
# Weights from model trained on training data provided by DhakaAI (round 1)
!gdown --id '1mq_SwpVeFULhchdJ5lZTUlfkHP2Czz_2'
%mv '/content/yolov5/weights_500epochs_yolov5x.pt' /content/yolov5/weights/myweights/


# Weights from models trained on training data provided by DhakaAI (round 1) & 
# custom data scraped from Google Images (link to dataset given above)

!gdown --id '1JZwqpPynxTudRKHFAL2_SW3VqYwHHqR_'
%mv '/content/yolov5/seed_1998_weights_100epochs_yolov5x.pt' /content/yolov5/weights/myweights/

!gdown --id '16CnpKQt2T3BiK46920gFDqH1pxBPjzA_'
%mv '/content/yolov5/seed_2900_weights_100epochs_yolov5x.pt' /content/yolov5/weights/myweights/

!gdown --id '111ygLTQq80_Kj6q-k6hsCvriww3crsgu'
%mv '/content/yolov5/seed1998_customdata_weights_finetune_100epochs.pt' /content/yolov5/weights/myweights/



# **** Round 2 ****
# Weights from models trained on training data provided by DhakaAI (round 1) & 
# custom data scraped from Google Images (link to dataset given above) &
# data collected from "Poribohon-BD" dataset (link to dataset given above)

!gdown --id '1X31QBI5gJuSGi1Nm0b4JfuURgh6nyM4z'
%mv '/content/yolov5/250epochs_d01_seed_3600.pt' /content/yolov5/weights/myweights/

!gdown --id '1Zbwj3nG08XPYMhBPEFht3ek8fJ65VEa_'
%mv '/content/yolov5/200_epochs_d04_seed_330.pt' /content/yolov5/weights/myweights/

!gdown --id '19coqIwF4VHa9GyIWpkJAs7n9KtAbrWU2'
%mv '/content/yolov5/150_epochs_d03_seed_900.pt' /content/yolov5/weights/myweights/

clear_output()




---


## 5. Submission

This part of the code deals with making the submission file. The actual yolo v3/v5 model outpus in the format  `(object-id) (score) (x-centre) (y-centre) (width) (height)` in each line for each object in the text label file. However, in the competition, we need to submit in this format `(image_name) (object_name) (xmin) (ymin) (xmax) (ymax) (width) (height)` for each line for each object in a `.csv` file. 


---



In [ ]:
%cd /content/yolov5/
clear_output()
import argparse
import os
import shutil
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords,
    xyxy2xywh, strip_optimizer, set_logging)
from utils.torch_utils import select_device, load_classifier, time_synchronized
from utils.plots import plot_one_box


def detect(save_img=False):
    out, source, weights, view_img, save_txt, imgsz = \
        opt.output, opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size
    webcam = source.isnumeric() or source.startswith(('rtsp://', 'rtmp://', 'http://')) or source.endswith('.txt')

    # Initialize
    set_logging()
    device = select_device(opt.device)
    if os.path.exists(out):
        shutil.rmtree(out)  # delete output folder
    os.makedirs(out)  # make new output folder
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    
    # if webcam part omitted previously
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    results=[]
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        t2 = time_synchronized()

        # Apply Classifier (removed in v3)
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            # if webcam part removed in v3
            if webcam:  # batch_size >= 1
                p, s, im0 = path[i], '%g: ' % i, im0s[i].copy()
            else:
                p, s, im0 = path, '', im0s

            save_path = str(Path(out) / Path(p).name)
            txt_path = str(Path(out) / Path(p).stem) + ('_%g' % dataset.frame if dataset.mode == 'video' else '')
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string
                
                xmin = []
                ymin = []
                xmax = []
                ymax = []
                scores = []
                labels_value=[]
                image_ids=[]                
                # Write results
                # reversed(det) --> det
                for *xyxy, conf, cls in det:
                    if save_txt:  # Write to file
                        conf_score = '%.2f' % (conf)
                        label_with_cls = '%s' % (names[int(cls)])
                        
                        labels_value.append(label_with_cls)
                        
                        xmin.append(int(xyxy[0]))
                        ymin.append(int(xyxy[1]))
                        xmax.append(int(xyxy[2]))
                        ymax.append(int(xyxy[3]))
                        
                        scores.append(conf_score)
                        image_ids.append(save_path)

                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        # line = (cls, conf, *xywh) if opt.save_conf else (cls, *xywh)  # label format
                        # with open(txt_path + '.txt', 'a') as f:
                        #     f.write(('%g ' * len(line) + '\n') % line)
                        with open(save_path[:save_path.rfind('.')] + '.txt', 'a') as file:
                            file.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format


                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                # else part removed in v3
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fourcc = 'mp4v'  # output video codec
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)
        result = {
            'image_id': image_ids,
            'score': scores,
            'class': labels_value,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax

            }

        results.append(result)
    if save_txt or save_img:
        print('Results saved to %s' % Path(out))

    print('Done. (%.3fs)' % (time.time() - t0))
    return results


In [ ]:
# Setting up parameters for inference /submission
class opt:
    cfg='/content/yolov5/models/yolov5x.yaml'
    weights = ['/content/yolov5/weights/myweights/150_epochs_d03_seed_900.pt','/content/yolov5/weights/myweights/200_epochs_d04_seed_330.pt','/content/yolov5/weights/myweights/250epochs_d01_seed_3600.pt','/content/yolov5/weights/myweights/seed1998_customdata_weights_finetune_100epochs.pt','/content/yolov5/weights/myweights/seed_1998_weights_100epochs_yolov5x.pt','/content/yolov5/weights/myweights/seed_2900_weights_100epochs_yolov5x.pt','/content/yolov5/weights/myweights/weights_500epochs_yolov5x.pt']
    
    # ****** NOTE TO ORGANIZERS ****** : 
    # MAKE SURE TO POINT TO THE DIRECTORY OF THE TEST SET
    # THAT WILL BE USED FOR CHECKING

    source='/content/test_round_2'        
    
    # ****** NOTE TO ORGANIZERS ****** : 
    # MAKE SURE TO POINT TO THE DIRECTORY OF THE TEST SET
    # THAT WILL BE USED FOR CHECKING
    
    save_txt=True
    output='output'  # Output directory of the results
    classes=False
    img_size=1024    # Inference Imag Size
    conf_thres=0.35
    iou_thres=0.6
    fourcc='mp4v'
    half=False
    device=''
    view_img=False
    agnostic_nms=False
    augment=True


In [ ]:
with torch.no_grad():
  res=detect()

Using torch 1.7.0+cu101 CUDA:0 (Tesla P4, 7611MB)



Fusing layers... 


Model Summary: 484 layers, 88525194 parameters, 0 gradients


Fusing layers... 


Model Summary: 484 layers, 88525194 parameters, 0 gradients


...

Fusing layers... 


Model Summary: 484 layers, 88525194 parameters, 0 gradients


Fusing layers... 


Model Summary: 484 layers, 88525194 parameters, 0 gradients


Ensemble created with ['/content/yolov5/weights/myweights/150_epochs_d03_seed_900.pt', '/content/yolov5/weights/myweights/200_epochs_d04_seed_330.pt', '/content/yolov5/weights/myweights/250epochs_d01_seed_3600.pt', '/content/yolov5/weights/myweights/seed1998_customdata_weights_finetune_100epochs.pt', '/content/yolov5/weights/myweights/seed_1998_weights_100epochs_yolov5x.pt', '/content/yolov5/weights/myweights/seed_2900_weights_100epochs_yolov5x.pt', '/content/yolov5/weights/myweights/weights_500epochs_yolov5x.pt']

image 1/450 /content/test_round_2/Asraf_100_jpg.rf.472d5bd950587612520051eebd827328.jpg: 1024x1024 2 buss, 1 minivans, 12 rickshaws, 1 three wheelers (CNG)s, Done. (4.726s)
image 2/450 /content/test_round_2/Asraf_101_jpg.rf.58690ebe018b849e3d3f6ef8785ca148.jpg: 1024x1024 3 buss, 2 cars, 1 motorbikes, 1 rickshaws, Done. (4.733s)
image 3/450 /content/test_round_2/Asraf_102_jpg.rf.40d695ed6f8db102530afcb11db466fa.jpg: 1024x1024 3 buss, 2 cars, 2 motorbikes, 4 rickshaws, Done. (



---


## 6. Writing the Submission File


---



In [ ]:
import pandas as pd
import numpy as np

append_data=[]

for i in range(len(res)):
  df = pd.DataFrame(res[i], columns = ['image_id','class','score','xmin','ymin','xmax','ymax'])
  append_data.append(df)

finl_results=pd.concat(append_data)
finl_results.image_id = [x.strip('output/') for x in finl_results.image_id]

finl_results['width'] = 1024
finl_results['height'] = 1024

print(f'Number of Objects Detected: {finl_results.shape[0]}')

# Saving the submission file with timestamp
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

finl_results.to_csv(f'{timestamp} submission.csv', index=False)
print('Submission file is written Successfully!')

finl_results.head()

Number of Objects Detected: 2654
Submission file is written Successfully!


,image_id,class,score,xmin,ymin,xmax,ymax,width,height
0,Asraf_100_jpg.rf.472d5bd950587612520051eebd827...,rickshaw,0.92,470,444,510,546,1024,1024
1,Asraf_100_jpg.rf.472d5bd950587612520051eebd827...,rickshaw,0.91,948,414,1024,568,1024,1024
2,Asraf_100_jpg.rf.472d5bd950587612520051eebd827...,rickshaw,0.91,427,437,464,540,1024,1024
3,Asraf_100_jpg.rf.472d5bd950587612520051eebd827...,three wheelers (CNG),0.91,518,425,606,631,1024,1024
4,Asraf_100_jpg.rf.472d5bd950587612520051eebd827...,rickshaw,0.83,646,424,692,572,1024,1024
